# WIP
# Pytorch
Na předchozích workshopech (resp. ve vedlejších noteboocích, pokud toto čtete na Githubu) jsme se věnovali základním technikám a algoritmům strojového učení. Ty měli výhodu nepříliš velkých hardwarových nároků a vcelku solidnímu výkonu na tabulkoidních datech. Nezmínili jsme tehdy ale neuronové sítě a deep learning. V povídání o textu jsem se o tyto termíny už trochu otřeli, nicméně na danou problematiku se pořádně podíváme až nyní.  
Technicky vzato jsou neuronové sítě jen podmnožinou strojového učení. Fakticky se ale jedná o velice důležitou podmnožinu, která s přehledem dokázala vyřešit problémy, na které klasické postupy nestačily - zmiňme třeba klasifikaci objektů na obrázcích či složitější práci s textem. Daní za tento výkon je větší složitost problematiky a hlavně větší doba trénování a hardwarové nároky.  
V současné době jsou v pythonu etablovány dva frameworky spojené s neuronovými sítěmi - TensorFlow (i s nadstavbou Keras) a PyTorch. Existují sice mezi nimi určité technické rozdíly, nicméně ty pro naše dnešní úvodní povídání nejsou zase až tak podstatné a navíc s v průběhu času postupně stírají. Když se podíváte na můj GitHub, uvidíte pár věcí založených na TensorFlowu a koneckonců nultou iteraci tohoto workshopu jsem ukazoval též na něm. Proč tedy v nadpisu vidíte PyTorch? Inu, důvod je dosti přizemní - Tensorflow 2.X není kompatibilní s grafickou kartou na mém notebooku, tudíž by i příprava ukázkových příkladů trvala příliš dlouho. No a ukazovat Vám příklady pro starou verzi Tensorflowu mi nepřijde jako úplně dobrý nápad. 

## Obsah

## Manipulace s tenzory  
Stejně jako balíček Pandas přináší dataframe (a sérii), na kterém poté staví celou svou činnost, nese s sebou Pytorch tenzor. Co to vlastně tenzor je? Tento termín se vyskytuje i mimo Pytorch a označuje se jím (matematici pominou zjednodušení) zobecnění skalárů, vektrorů a matic do vyšších dimenzí. Člověk by si mohl myslet, že stejnou funkčnost zastanou stávající pythoní objekty a není třeba vymýšlet něco nového. Bohužel je ale situace komplikovanější. V Pythonu je totiž každá věc včetně integerů či floatů objekt. To znamená, že tyto konstrukce obsahují krom číslené informace i něco navíc. Tohle "něco" sice přináší dodatečnou funkčnost, ale také to zabírá víc paměti. U pár čísel to nevadí, jenomže při práci s neuronovými sítěmi s pouhými několika čísli pracovat nebudeme. No a výhodou tenzoru je to, že čísla v něm obsažená jsou opravdu jen čísla bez čehokoli dalšího. Jinak pytorchí tenzory mohou obsahovat pouze čísla (pokud booleany budeme brát také za čísla), nikoli ale textové řetězce.  
No a jelikož jsou tenzory základním stavebním kamenem Pytorche, měli bychom se s nimi naučit pracovat, než budeme dělat cokoli komplikovanějšího.  

Z historických důvodů se importovaný balíček nejmenuje pytorch, nýbrž torch.

In [1]:
import torch

Asi nejpřímočařejší je vytvoření 1D tenzorů obsahující pouze jedničk či nuly. Parametr funkcí *ones* a *zeros* udává počet těchto čísel.

In [2]:
tensor_of_ones = torch.ones(5)
print("Tensor of ones:")
print(tensor_of_ones)

tensor_of_zeros = torch.zeros(7)
print ("Tensor of zeros:")
print(tensor_of_zeros)

Tensor of ones:
tensor([1., 1., 1., 1., 1.])
Tensor of zeros:
tensor([0., 0., 0., 0., 0., 0., 0.])


K elementům tenzoru můžeme přistupovat "normálně", tj přes index. Za pozornost asi stojí zmínka, že i když chceme jen jeden element tenzoru, nevrátí se nám číslo, ale tenzor.

In [3]:
tensor_of_ones[0]

tensor(1.)

Tenzory nejsou immutable, nýbrž se čísla v nich dají měnit:

In [4]:
tensor_of_ones[-1] = 999
tensor_of_ones

tensor([  1.,   1.,   1.,   1., 999.])

Pokud chceme mít tenzor jedniček/nul o více rozměrech, napíšeme zkrátka do funkce více parametrů.

In [5]:
torch.zeros(2,3)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [6]:
torch.zeros(2,3, 4)

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

Obecný tenzor se dá vyrobit například pomocí listu

In [7]:
tensor_of_numbers = torch.tensor([1,2,3,4,5,6])
tensor_of_numbers

tensor([1, 2, 3, 4, 5, 6])

Takto lze vytvořit i vícedimenzionální tenzory - v listu musí být podlisty

In [8]:
tensor_more_dims = torch.tensor([
    [1, 2],
    [3, 4],
    [5, 6]
])
tensor_more_dims

tensor([[1, 2],
        [3, 4],
        [5, 6]])

Upozorněme, že nelze vytvořit "pilovitý tenzor", tj. něco v duchu
```python
tensor_bad = torch.tensor([
    [1, 2],
    [3],
    [5, 6]
])
```
Tvar (velikost tenzoru v jednotlivých dimenzích) se získá pomocí atributu shape

In [9]:
print(f"Shape of 1D tensor: {tensor_of_numbers.shape}")
print(f"Shape of 2D tensor: {tensor_more_dims.shape}")

Shape of 1D tensor: torch.Size([6])
Shape of 2D tensor: torch.Size([3, 2])


Když hovoříme o dimenzích, asi bychom měli zmínit funkce squeeze a unsqueeze. Squeeze z tenzoru odstraní všechny dimenze o velikosti 1.

In [10]:
tensor_before_squeeze = torch.ones(2,1)
print("Tensor before squeze:")
print(tensor_before_squeeze)
print(f"Shape of tensor before squeeze: {tensor_before_squeeze.shape}\n")

tensor_after_squeeze = tensor_before_squeeze.squeeze()
print("Tensor after squeeze:")
print(tensor_after_squeeze)
print(f"Shape of tensor after squeeze: {tensor_after_squeeze.shape}\n")

Tensor before squeze:
tensor([[1.],
        [1.]])
Shape of tensor before squeeze: torch.Size([2, 1])

Tensor after squeeze:
tensor([1., 1.])
Shape of tensor after squeeze: torch.Size([2])



Unsqueeze naopak na místo určené uživatelem dimenzi o velikosti jedna přidá. Typické užití je v převedení jednoho tenzoru do stejného rozměru, jako má tenzor jiný, aby se s oběma součaasně dalo pracovat.

In [11]:
tensor_before_unsqueeze = torch.ones(2,2)
print("Tensor before unsqueze:")
print(tensor_before_unsqueeze)
print(f"Shape of tensor before unsqueeze: {tensor_before_unsqueeze.shape}\n")

tensor_after_unsqueeze = tensor_before_unsqueeze.unsqueeze(1)
print("Tensor after unsqueeze:")
print(tensor_after_unsqueeze)
print(f"Shape of tensor after unsqueeze: {tensor_after_unsqueeze.shape}\n")

Tensor before unsqueze:
tensor([[1., 1.],
        [1., 1.]])
Shape of tensor before unsqueeze: torch.Size([2, 2])

Tensor after unsqueeze:
tensor([[[1., 1.]],

        [[1., 1.]]])
Shape of tensor after unsqueeze: torch.Size([2, 1, 2])



Výše jsme si ukázali, jak se dostat k číslu v 1D tenzoru. U vícedimenzionálních tenzorů přistupujeme k jednotlivým číslům pomocí n indexů, kde n je počet dimenzí.

In [12]:
tensor_more_dims[1,0]

tensor(3)

Případně můžeme vzít celý řádek

In [13]:
row = tensor_more_dims[1]
row

tensor([3, 4])

Či sloupec

In [14]:
column = tensor_more_dims[:,1]
column

tensor([2, 4, 6])

Bacha ale, takhle se nevyrání nové tenzory, pouze se na původní data koukáme jiným způsobem.

In [15]:
column[0] = 20
tensor_more_dims

tensor([[ 1, 20],
        [ 3,  4],
        [ 5,  6]])

Fakticky to funguje tak, že jsou data v tzv storagi - vždy jednorozměrném poli. Tenzory jsou pak jenom pohledem na toto pole. Narozdíl od tenzoru nemá storage parametr shape.

In [16]:
tensor_more_dims.storage()

 1
 20
 3
 4
 5
 6
[torch.LongStorage of size 6]

Funkce storage_offset() vrací index, od kterého ve storagi začíná první prvek příslušného tenzoru.

In [17]:
tensor_more_dims.storage_offset()

0

In [18]:
column.storage_offset()

1

In [19]:
row.storage_offset()

2

Stride zase říká, kolik prvků se musí ve storagi přeskočit, když se v tenzoru přesuneme o jednu pozici. Výsledný tuple ukazuje stride pro každou dimenzi.

In [20]:
tensor_more_dims.stride()

(2, 1)

In [21]:
column.stride()

(2,)

In [22]:
row.stride()

(1,)

Proč vůbec o těchto věcech mluvíme? Díky nim jsou mnohé operace s tenzory výpočetně nenáročné - nedochází totiž k přeuspořádání dat, pouze se změní několik málo metadat. Příkladem takovéto operace může být transpozice, tj. prohození sloupců a řádků. Realizujeme ji pomocí metody *t*.

In [23]:
original_tensor = torch.tensor([
    [1, 2],
    [3, 4],
    [5, 6],
])
print("Original tensor")
print(original_tensor)
print(f"Original tensor metadata -  offset: {original_tensor.storage_offset()}, stride: {original_tensor.stride()}\n")
print("Transponed tensor")
transponed_tensor = original_tensor.t()
print(transponed_tensor)
print(f"Transponed tensor metadata -  offset: {transponed_tensor.storage_offset()}, stride: {transponed_tensor.stride()}\n")

Original tensor
tensor([[1, 2],
        [3, 4],
        [5, 6]])
Original tensor metadata -  offset: 0, stride: (2, 1)

Transponed tensor
tensor([[1, 3, 5],
        [2, 4, 6]])
Transponed tensor metadata -  offset: 0, stride: (1, 2)



Transpozici lze realizovat i na tenzorech o vyšších dimenzích. Tehdy se musí použít funkce *transpose*, která přebírá dva parametry - indexy prohazovaných os. Tj. pro 2D tenzory je metodě *t* ekvivaletní *transpose(0,1)*.

In [24]:
many_dim_tensor = torch.zeros(2,3,4)
print(many_dim_tensor.shape)
transponed_many_dim_tensor = many_dim_tensor.transpose(1,2)
print(transponed_many_dim_tensor.shape)

torch.Size([2, 3, 4])
torch.Size([2, 4, 3])


Existuje ale ještě jedna, asi dokonce častěji používaná funckionalita - pohledy. O co se jedná? Dalo by se říci, že pohled se má k tenzoru jako tenzor ke storagi. Jinými slovy jedná se o způsob, jak přespořádat elementy tenzoru. Z hlediska technického je view metoda tenzoru, která přebírá n argumentů. Ty popořadě specifikují velikost dimenze, jejíž index odpovídá pořadí argumentu. Pokud člověk na nějaké místo (nejen na konec, jako v příkladu, ale i kamkoli jinam) napíše mínus jedničku, dopočítá se velikost dimenze z ostatních dimenzí a počtu dat v původním tenzoru.

In [25]:
normal_tensor = torch.tensor([
    [1,2,3,4,5,6],
    [7,8,9,10,11,12]
])

first_view = normal_tensor.view(3,4)
second_view = normal_tensor.view(2,3,-1)

print("Original tensor:")
print(normal_tensor)
print("First tensor:")
print(first_view)
print("Second tensor:")
print(second_view)

Original tensor:
tensor([[ 1,  2,  3,  4,  5,  6],
        [ 7,  8,  9, 10, 11, 12]])
First tensor:
tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])
Second tensor:
tensor([[[ 1,  2],
         [ 3,  4],
         [ 5,  6]],

        [[ 7,  8],
         [ 9, 10],
         [11, 12]]])


Někdy potřebujeme udělat kopii tenzoru, která nebude s originálem sdílet data ve storagi. Tehdy musíme použít metodu *clone*.

In [26]:
cloned_tensor = original_tensor.clone()
cloned_tensor[0,0] = 50
print("Original tensor")
print(original_tensor)
print("Cloned tensor")
print(cloned_tensor)

Original tensor
tensor([[1, 2],
        [3, 4],
        [5, 6]])
Cloned tensor
tensor([[50,  2],
        [ 3,  4],
        [ 5,  6]])


Pro porovnání elementů v tenzoru s určitou hodnotou můžeme použít následující funkce:
- lt (neboli lower than)
- le (neboli lower than or equal)
- eq (neboli equal)
- ge (neboli greater than or equal)
- gt (neboli greather than) 

Jejich samostatné použití vyústí v tenzor booleanů:

In [27]:
comparison_tensor = torch.tensor([
    [1,2,3],
    [4,5,6],
    [7,8,9],
    [10,11,12]
])
print("Original tensor")
print(comparison_tensor)
print("Tensor of comparison validity:")
print(comparison_tensor.eq(5))

Original tensor
tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]])
Tensor of comparison validity:
tensor([[False, False, False],
        [False,  True, False],
        [False, False, False],
        [False, False, False]])


Pro zjištění hodnot podmínku splňující musíme podobně jako v pandách použít hranaté závorky.

In [28]:
comparison_tensor[comparison_tensor.ge(5)]

tensor([ 5,  6,  7,  8,  9, 10, 11, 12])

Pokud chceme v jeden okamžik použít více podmínek, musíme je svázat pomocí & (má význam *and*) či | (má význam *or*)

In [29]:
comparison_tensor[comparison_tensor.lt(3) | comparison_tensor.gt(10)]

tensor([ 1,  2, 11, 12])

In [30]:
comparison_tensor[comparison_tensor.gt(3) & comparison_tensor.lt(10)]

tensor([4, 5, 6, 7, 8, 9])

Další pro obálkoidní objekty (listy, dataframy) typickou úlohou je nalepování jeden na druhý. To se v Pytorchi realizuje funkcí cat. Jejím prvním parametrem je tuple či list s na sebe nalepovanými tenzory, druhý parametr - dim - pak říká, přes jakou osu/dimezi se na sebe tenzory vlastě mají lepit. Platí přitom, že ostatní dimeze tenzorů by měly mít stejný rozměr.

In [31]:
first_tensor = torch.tensor([
    [1,2,3,4],
    [5,6,7,8]
])
second_tensor = torch.tensor([
    [9,10,11,12]
])
concatenated_tensor = torch.cat((first_tensor, second_tensor), dim=0)
concatenated_tensor

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])

Jakých datových typů vlastně mohou hodnoty v tenzoru nabývat? Máme k dispozici několikero typů floatů a integerů + booleany:
- torch.bool  
- torch.float16 (alias torch.half)
- torch.float32 (alias torch.float) - jedná se o default
- torch.float64 (alias torch.double)
- torch.int8
- torch.uint8
- torch.int16 (alias torch.short)
- torch.int32 (alias torch.int)
- torch.int64 (alias torch.long)  - jedná se default při vložení celých čísel do kontruktoru
  
Datový typ můžeme nastavit při vytvoření tenzoru s pomocí parametru dtype:

In [32]:
tensor_integers = torch.tensor([1,2,3,4], dtype=torch.int32)
tensor_integers

tensor([1, 2, 3, 4], dtype=torch.int32)

In [33]:
tensor_floats = torch.tensor([1,2,3,4], dtype=torch.float32)
tensor_floats

tensor([1., 2., 3., 4.])

Pro konverzi již existujících tensorů lze použít metody *to* či *type*. Pro jistotu zdůrazněme, že původní tenzor si zachovává svůj datový typ.

In [34]:
tensor_float64 = tensor_integers.to(dtype=torch.double)
tensor_float64

tensor([1., 2., 3., 4.], dtype=torch.float64)

In [35]:
tensor_int16 = tensor_floats.type(torch.short)
tensor_int16

tensor([1, 2, 3, 4], dtype=torch.int16)

S pytorchími tenzory a numpoidními poli se pracuje dosti podobně a tak není moc překvapivé, že jedny můžeme konvertovat na druhé. Přeměna tezoru na numpy array se provede pomocí metody *numpy*.

In [36]:
some_tensor = torch.tensor([1,2,3,4])
some_numpy_array = some_tensor.numpy()
some_numpy_array

array([1, 2, 3, 4], dtype=int64)

Opačný proces se realizuje funkcí *from_numpy*.

In [37]:
tensor_from_numpy = torch.from_numpy(some_numpy_array)
print(tensor_from_numpy)
print(tensor_from_numpy.dtype)

tensor([1, 2, 3, 4])
torch.int64


V kontextu deep learningu je významný přesun tenzorů z CPU na GPU. Proč vlastně GPU vůbec řešíme? Jeden výpočetní element (jádro) na CPU je silnější než jeden výpočetní element na GPU. Jenomže zatímco jader na CPU máme pár jednotek, na GPU je jich řádově mnohem více. No a vzhledem k tomu, že výpočty spojené s neuronovými sitěmi se dají dobře paralelizovat, trvají operace s použitím GPU mnohem kratší dobu. Jenže aby nějaký výpočet na GPU mohl probíhat, musíme tam nejprve dostat vstupní data.  
Jedním ze způsobů, jak toho docílit, je stanovit už v konstruktoru tenzoru, že mají operace probíhat na *device="cuda"*. CUDA (Compute Unified Device Architecture) je platforma pro paralelní výpočty nad grafickými kartami Nvidie. Pozn.: nula v outputu buňky níže označuje index grafické karty.

In [38]:
tensor_gpu = torch.tensor([1,2,3,4], device="cuda")
tensor_gpu

tensor([1, 2, 3, 4], device='cuda:0')

Už vytvořené tenzory zkopírujeme na GPU pomocí metody *to*. Stejná metoda pak zase zkopíruje tenzor z GPU na CPU.

In [39]:
tensor_cpu = torch.tensor([1,2,3,4])
tensor_moved_to_gpu = tensor_cpu.to(device="cuda")
tesor_moved_to_cpu = tensor_moved_to_gpu.to(device="cpu")

print(tensor_cpu)
print(tensor_moved_to_gpu)
print(tesor_moved_to_cpu)

tensor([1, 2, 3, 4])
tensor([1, 2, 3, 4], device='cuda:0')
tensor([1, 2, 3, 4])


Zdůrazněme, že se tu opravdu jedná o kopírování, tj. změna v původním objektu nevyvolá změnu v objektu na novém působišti.

In [40]:
tensor_cpu[0] = 10
tensor_moved_to_gpu[1] = 20
tesor_moved_to_cpu[2] = 30

print(tensor_cpu)
print(tensor_moved_to_gpu)
print(tesor_moved_to_cpu)

tensor([10,  2,  3,  4])
tensor([ 1, 20,  3,  4], device='cuda:0')
tensor([ 1,  2, 30,  4])


Co dělat, když chceme tenzor uložit na disk? Nejpřímočařejší je využití funkce *save*, která fakticky objekt uloží ve formátu pickle.

In [41]:
saved_tensor = torch.tensor([1,2,3,4])
torch.save(saved_tensor, "saved_tensor.t")

Nahrát tenzor z disku můžeme pomocí funkce load.

In [42]:
loaded_tensor = torch.load("saved_tensor.t")
loaded_tensor

tensor([1, 2, 3, 4])

Možností pro ukládání je více - například lze použít formát HDF5. To zde ale ukazovat nebudeme.